# 1. Modulos

In [ ]:
import os
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import openpyxl

# 2. Rutas 

In [ ]:
# Validados
R_NS = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\NS\NS validado\all-NS-VAL.txt"
#R_NS = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\NS\NS validado\all-NS-VAL.txt"
#R_NS_neg = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\NS\NS Negado\all-NS-NEG.txt"
# Glosados
R_NS_neg = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\NS\NS Negado\all-NS-NEG.txt"

# 3. Cargue Datafarmes 

In [ ]:
df_ns = pd.read_csv(R_NS, sep=',', dtype=str, encoding='latin-1')
df_ns_neg = pd.read_csv(R_NS_neg, sep=',', dtype=str, encoding='latin-1')
# Convertir Fecha_Proceso a datetime y filtrar desde enero 2024
df_ns['Fecha_Proceso'] = pd.to_datetime(df_ns['Fecha_Proceso'], format='%d/%m/%Y')
df_ns = df_ns[df_ns['Fecha_Proceso'] >= '2022-01-01']
df_ns = df_ns.sort_values(by='NOVEDAD', ascending=False)

# 4. Validar
## 4.1. Columnas

In [ ]:
df_ns.columns

## 4.2. Numero de registros por novedad

In [ ]:
df_ns['NOVEDAD'].value_counts().sort_index()

## 4.3. N14 Actualización o cambio de estado de afiliación

In [ ]:
# Filtrar registros donde NOVEDAD es N14
df_n14 = df_ns[df_ns['NOVEDAD'] == 'N14']

# Obtener un registro por cada grupo de COD_1_NOVEDAD
muestra_n14 = df_n14.groupby('COD_1_NOVEDAD').first().reset_index()

print(f"Total de grupos distintos en COD_1_NOVEDAD para N14: {len(muestra_n14)}")
print("\nRegistros de muestra (1 por cada COD_1_NOVEDAD):")
print(muestra_n14)

## 4.4. N17 filtro

In [ ]:
# Filtrar registros donde NOVEDAD es N17
df_n17 = df_ns[df_ns['NOVEDAD'] == 'N17']

## 4.5. N21 filtro

In [ ]:
# Filtrar registros donde NOVEDAD es N21 y COD_3_NOVEDAD es diferente a '5'
df_n21 = df_ns[(df_ns['NOVEDAD'] == 'N21') & (df_ns['COD_3_NOVEDAD'] != '5')]

# 5. Estadisticas
## 5. novedades totales por año 

In [ ]:
# Extraer el año de Fecha_Proceso y contar registros por año
registros_por_año = df_ns['Fecha_Proceso'].dt.year.value_counts().sort_index()

# Crear una tabla comparativa
tabla_años = pd.DataFrame({
    'Año': registros_por_año.index,
    'Total Registros': registros_por_año.values
})

print("Tabla de registros por año:")
print(tabla_años)
print(f"\nTotal general: {tabla_años['Total Registros'].sum():,} registros")

# Crear gráfica
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(registros_por_año.index, registros_por_año.values, color='steelblue', edgecolor='black')
plt.xlabel('Año', fontsize=12)
plt.ylabel('Cantidad de Registros', fontsize=12)
plt.title('Cantidad de Registros Totales por Año', fontsize=14, fontweight='bold')
plt.xticks(registros_por_año.index)
plt.grid(axis='y', alpha=0.3)

# Añadir valores encima de las barras
for año, cantidad in zip(registros_por_año.index, registros_por_año.values):
    plt.text(año, cantidad, f'{cantidad:,}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd

# --- 1. Configuración de Estilo y Datos ---
sns.set_context("notebook", font_scale=1.1)
sns.set_style("white") # Fondo blanco es más limpio para informes que el grid gris por defecto

# Extraer mes y año (Mantenemos tu lógica original)
df_ns['Mes'] = df_ns['Fecha_Proceso'].dt.month
df_ns['Año'] = df_ns['Fecha_Proceso'].dt.year
df_ns['Año_Mes'] = df_ns['Fecha_Proceso'].dt.to_period('M')

# Crear tabla pivote
registros_mes_año = pd.crosstab(df_ns['Mes'], df_ns['Año'], margins=True, margins_name='Total')

print("Resumen de Registros por Mes y Año:")
display(registros_mes_año) # 'display' es mejor que print en notebooks

# Paleta personalizada (Tus colores)
# Nota: Si el año 2025 está incompleto, visualmente se notará por la caída a cero si no hay datos.
colores_personalizados = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']
nombres_meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 
                 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

# --- 2. Gráfica de Líneas (Evolución Temporal) ---
plt.figure(figsize=(16, 8))
ax = plt.gca()

# Iterar por años para graficar
years = sorted(df_ns['Año'].unique())
for idx, anio in enumerate(years):
    # Filtrar datos
    datos_anio = df_ns[df_ns['Año'] == anio].groupby('Mes').size()
    
    # Reindexar para asegurar que aparezcan todos los meses (1-12) aunque sean 0
    datos_anio = datos_anio.reindex(range(1, 13), fill_value=0)
    
    # Estilo de línea: Si es el año actual (ej. 2025), a veces se usa punteado, 
    # pero mantendremos sólido por consistencia.
    color_actual = colores_personalizados[idx % len(colores_personalizados)]
    
    plt.plot(datos_anio.index, datos_anio.values,
             marker='o', 
             linewidth=2.5,
             color=color_actual,
             markersize=8,
             markeredgecolor='white',
             markeredgewidth=1.5,
             label=f'{anio}')
    
    # Opcional: Anotar el valor máximo de cada año para dar contexto rápido
    max_val = datos_anio.max()
    max_month = datos_anio.idxmax()
    if max_val > 0:
        plt.annotate(f'{max_val:,.0f}', 
                     (max_month, max_val), 
                     textcoords="offset points", 
                     xytext=(0,10), 
                     ha='center', 
                     fontsize=9, 
                     color=color_actual,
                     fontweight='bold')

# Configuración de Ejes y Títulos
plt.title('Evolución Mensual de Registros por Año', fontsize=18, fontweight='bold', pad=20, loc='left')
plt.xlabel('Mes del Proceso', fontsize=12, fontweight='bold', color='#555555')
plt.ylabel('Cantidad de Registros', fontsize=12, fontweight='bold', color='#555555')

# Eje X: Meses
plt.xticks(ticks=range(1, 13), labels=nombres_meses, fontsize=11)

# Eje Y: Formato de miles (ej: 10,000)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

# Limpieza visual (Eliminar bordes innecesarios)
sns.despine() 
plt.grid(axis='y', linestyle='--', alpha=0.3) # Solo grid horizontal suave

# Leyenda limpia
plt.legend(title='Año', title_fontsize=12, fontsize=11, frameon=False, loc='upper left')

plt.tight_layout()
plt.show()

# --- 3. Heatmap (Mapa de Calor) ---
plt.figure(figsize=(14, 8))

# Datos para heatmap (Excluyendo totales para no sesgar la escala de colores)
datos_heatmap = registros_mes_año.iloc[:-1, :-1]

# Graficar
# Sugerencia: Usar una paleta secuencial como "YlGnBu" o "mako" es más profesional para volúmenes.
# "RdYlGn_r" implica semántica (Rojo=Malo/Alto), úsala solo si muchos registros es algo negativo.
ax_heat = sns.heatmap(datos_heatmap, 
                      annot=True, 
                      fmt=',d',  # Agrega separador de miles (Python 3)
                      cmap='YlGnBu', # Paleta profesional azul-verdosa
                      linewidths=1, 
                      linecolor='white',
                      cbar_kws={'label': 'Volumen de Registros'},
                      annot_kws={'size': 10})

# Ajustes finales del Heatmap
plt.title('Concentración de Registros: Mes vs Año', fontsize=16, fontweight='bold', pad=20, loc='left')
plt.ylabel('Mes', fontsize=12, fontweight='bold')
plt.xlabel('Año', fontsize=12, fontweight='bold')

# Asegurar etiquetas correctas en Eje Y
ax_heat.set_yticklabels(nombres_meses, rotation=0)

plt.tight_layout()
plt.show()

## Novedades por categorias año a año

In [ ]:
# Crear grupos de novedades basados en el prefijo (N01, N02, etc.)
df_ns['Grupo_Novedad'] = df_ns['NOVEDAD'].str[:3]
df_ns['Año'] = df_ns['Fecha_Proceso'].dt.year

# Crear tabla comparativa de registros por grupo de novedad y año
tabla_comparativa = pd.crosstab(df_ns['Grupo_Novedad'], df_ns['Año'], margins=True, margins_name='Total')
tabla_comparativa = tabla_comparativa.sort_index()

print("Tabla comparativa de registros por grupo de novedad y año:")
print(tabla_comparativa)
print("\n")

# Crear gráfica de líneas para ver la evolución temporal
import matplotlib.pyplot as plt

# Preparar datos para la gráfica (sin los totales)
datos_grafica = pd.crosstab(df_ns['Grupo_Novedad'], df_ns['Año'])

plt.figure(figsize=(14, 8))

# Graficar cada grupo de novedad
for grupo in datos_grafica.index:
    plt.plot(datos_grafica.columns, datos_grafica.loc[grupo], marker='o', label=grupo, linewidth=2)

plt.xlabel('Año', fontsize=12)
plt.ylabel('Cantidad de Registros', fontsize=12)
plt.title('Evolución de Registros por Grupo de Novedad a lo Largo del Tiempo', fontsize=14, fontweight='bold')
plt.legend(title='Grupo Novedad', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Crear gráfica de barras apiladas para comparación visual
plt.figure(figsize=(14, 8))

datos_grafica.T.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='tab20')
plt.xlabel('Año', fontsize=12)
plt.ylabel('Cantidad de Registros', fontsize=12)
plt.title('Cantidad de Registros por Grupo de Novedad y Año (Barras Apiladas)', fontsize=14, fontweight='bold')
plt.legend(title='Grupo Novedad', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Validadas VS NEgadas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import seaborn as sns

# --- 1. Configuración ---
sns.set_style("white")
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

# --- 2. Preparación de Datos ---
df_ns = df_ns.copy()
df_ns_neg = df_ns_neg.copy()
df_ns['Tipo'] = 'Validada'
df_ns_neg['Tipo'] = 'Negada'

# Conversión y limpieza
df_ns['Fecha_Proceso'] = pd.to_datetime(df_ns['Fecha_Proceso'], format='%d/%m/%Y', errors='coerce')
df_ns_neg['Fecha_Proceso'] = pd.to_datetime(df_ns_neg['Fecha_Proceso'], format='%d/%m/%Y', errors='coerce')

df_comparativo = pd.concat([df_ns[['Fecha_Proceso', 'Tipo']], df_ns_neg[['Fecha_Proceso', 'Tipo']]], ignore_index=True)
df_comparativo['Año'] = df_comparativo['Fecha_Proceso'].dt.year
df_comparativo['Mes_Num'] = df_comparativo['Fecha_Proceso'].dt.month

efectividad_anual = df_comparativo.groupby(['Año', 'Mes_Num', 'Tipo']).size().unstack(fill_value=0)

for col in ['Validada', 'Negada']:
    if col not in efectividad_anual.columns:
        efectividad_anual[col] = 0

efectividad_anual['Total'] = efectividad_anual['Validada'] + efectividad_anual['Negada']
efectividad_anual['Tasa_Efectividad'] = np.where(
    efectividad_anual['Total'] > 0, (efectividad_anual['Validada'] / efectividad_anual['Total']) * 100, 0
)

# --- 3. Dashboard con "Peso Visual" ---

años_unicos = sorted(efectividad_anual.index.get_level_values(0).unique())
num_años = len(años_unicos)
ncols = 3
nrows = int(np.ceil(num_años / ncols))

# CALCULAR EL MÁXIMO GLOBAL DE VOLUMEN (Para calibrar la intensidad del color)
# Buscamos el mes con más registros en toda la historia para usarlo de referencia 100%
max_volumen_mensual_historico = efectividad_anual['Validada'].max()

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 6 * nrows), facecolor='#f8f9fa')
axes = axes.flatten()

# Colores
color_validada = '#2ca02c' 
color_negada = '#d62728'   
color_tasa = '#1f77b4'     
color_kpi = '#7f7f7f'

meses_labels = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

for i, ax in enumerate(axes):
    if i < num_años:
        año = años_unicos[i]
        datos_año = efectividad_anual.loc[año].reindex(range(1, 13), fill_value=0)
        
        # Totales para el título
        total_validada_año = datos_año['Validada'].sum()
        total_total_año = datos_año['Total'].sum()
        promedio_anual = (total_validada_año / total_total_año * 100) if total_total_año > 0 else 0
        
        # --- LÓGICA DE INTENSIDAD VISUAL (OPACIDAD) ---
        # Calculamos qué tan "grande" es el pico de este año comparado con el histórico
        max_volumen_este_año = datos_año['Validada'].max()
        
        # Factor de opacidad: Mínimo 0.05 (para que se vea algo) hasta 0.6 (sólido)
        # Si el año es pequeño, ratio será bajo -> opacidad baja.
        ratio_volumen = max_volumen_este_año / max_volumen_mensual_historico
        alpha_fill = 0.05 + (0.55 * ratio_volumen) 
        
        # --- GRÁFICO ---
        
        # 1. Área Rellena (Visual Weight): Indica el volumen relativo
        ax.fill_between(datos_año.index, datos_año['Validada'], color=color_validada, alpha=alpha_fill)
        
        # 2. Líneas Principales (Tendencia)
        ax.plot(datos_año.index, datos_año['Validada'], 
                color=color_validada, marker='o', markersize=4, linewidth=2, label='Validada')
        
        ax.plot(datos_año.index, datos_año['Negada'], 
                color=color_negada, marker='x', markersize=4, linewidth=1.5, linestyle=':', label='Negada')

        # Título enriquecido con VOLUMEN TOTAL
        title_color = 'black' if ratio_volumen > 0.1 else '#666666' # Título gris si es un año irrelevante
        
        ax.set_title(f'Año {año}\nVol Total: {int(total_total_año):,} | Efec: {int(promedio_anual)}%', 
                     fontsize=14, fontweight='bold', color=title_color, pad=15)
        
        ax.set_xticks(range(1, 13))
        ax.set_xticklabels(meses_labels, fontsize=10)
        ax.yaxis.set_major_formatter(ticker.EngFormatter())
        ax.grid(True, axis='y', linestyle='-', alpha=0.3)

        # Eje Derecho: Tasa
        ax2 = ax.twinx()
        ax2.axhline(y=95, color=color_kpi, linestyle='--', linewidth=1, alpha=0.5) # Meta
        
        ax2.plot(datos_año.index, datos_año['Tasa_Efectividad'], 
                 color=color_tasa, marker='D', markersize=4, linewidth=1, linestyle='-', alpha=0.7, label='Tasa (%)')
        
        # Etiquetas de Tasa (Data Labels)
        for x, y, total in zip(datos_año.index, datos_año['Tasa_Efectividad'], datos_año['Total']):
            if total > 0:
                offset = 7 if y < 90 else -12 
                va_align = 'bottom' if y < 90 else 'top'
                ax2.annotate(f'{int(y)}%', (x, y), textcoords="offset points", xytext=(0, offset), 
                             ha='center', va=va_align, fontsize=9, color=color_tasa, fontweight='bold',
                             bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7))

        ax2.set_ylim(0, 115)
        ax2.yaxis.set_major_formatter(ticker.PercentFormatter())
        ax2.tick_params(axis='y', colors=color_tasa)
        
        ax.spines['top'].set_visible(False)
        ax2.spines['top'].set_visible(False)

    else:
        ax.axis('off')

# Leyenda
lines_1, labels_1 = axes[0].get_legend_handles_labels()
lines_2, labels_2 = axes[0].twinx().get_legend_handles_labels()
fig.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper center', bbox_to_anchor=(0.5, 0.94), ncol=4, frameon=False)

fig.suptitle('Dashboard de Efectividad e Impacto Operativo (Intensidad de Color = Volumen Relativo)', 
             fontsize=18, fontweight='bold', y=0.98)

plt.tight_layout(rect=[0, 0, 1, 0.92])
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 

# --- 1. Preparación de DataFrames y Conversión de Fechas ---
# ... (Este bloque permanece sin cambios) ...
df_ns['Tipo'] = 'Validada'
df_ns_neg['Tipo'] = 'Negada'

df_ns['Fecha_Proceso'] = pd.to_datetime(df_ns['Fecha_Proceso'], format='%d/%m/%Y')
df_ns_neg['Fecha_Proceso'] = pd.to_datetime(df_ns_neg['Fecha_Proceso'], format='%d/%m/%Y')

df_comparativo = pd.concat([df_ns[['Fecha_Proceso', 'Tipo']], df_ns_neg[['Fecha_Proceso', 'Tipo']]], ignore_index=True)

# --- 2. Preparación para la Comparación Anual (Mes vs. Año) ---
# ... (Este bloque permanece sin cambios) ...
df_comparativo['Año'] = df_comparativo['Fecha_Proceso'].dt.year
df_comparativo['Mes_Num'] = df_comparativo['Fecha_Proceso'].dt.month
df_comparativo['Mes_Label'] = df_comparativo['Fecha_Proceso'].dt.strftime('%b') 

efectividad_anual_comp = df_comparativo.groupby(['Año', 'Mes_Num', 'Mes_Label', 'Tipo']).size().unstack(fill_value=0).reset_index()

efectividad_anual_comp['Tasa_Efectividad'] = (efectividad_anual_comp['Validada'] / (efectividad_anual_comp['Validada'] + efectividad_anual_comp['Negada'])) * 100

# --- 3. Generación del Gráfico Limpio de Tasa de Efectividad Mes a Mes (CON ETIQUETAS) ---

años = sorted(efectividad_anual_comp['Año'].unique())
num_años = len(años)
# Identificar el año más reciente para aplicar etiquetas (último en la lista ordenada)
año_reciente = años[-1]

colores = plt.cm.tab10(np.linspace(0, 1, num_años))
meses_labels = [pd.to_datetime(str(m), format='%m').strftime('%b') for m in range(1, 13)]


plt.figure(figsize=(14, 7))
ax = plt.gca()

# Iterar sobre cada año para graficar
for i, año in enumerate(años):
    df_año = efectividad_anual_comp[efectividad_anual_comp['Año'] == año]
    color_line = colores[i]
    
    # Graficar la Tasa de Efectividad
    ax.plot(
        df_año['Mes_Num'], 
        df_año['Tasa_Efectividad'], 
        color=color_line, 
        marker='o', 
        linestyle='-', 
        linewidth=2,
        label=f'Tasa Efectividad {año}'
    )
    
    # 💡 IMPLEMENTACIÓN DE ETIQUETAS: Solo para el año más reciente (2025)
    if año == año_reciente:
        for x, y in zip(df_año['Mes_Num'], df_año['Tasa_Efectividad']):
            # Usar una pequeña compensación vertical (y+2) para que la etiqueta no tape el punto
            ax.text(
                x, 
                y + 2, 
                f'{y:.1f}%', # Formatear a un decimal
                fontsize=9, 
                color=color_line,
                ha='center', # Alineación horizontal centrada
                fontweight='bold'
            )
    
# --- Configuración Final del Gráfico ---
ax.set_xlabel('Mes del Año')
ax.set_ylabel('Tasa de Efectividad (%)')
ax.set_title('Comparación Anual de Tasa de Efectividad (Mes a Mes)')

# Configurar Eje X
ax.set_xticks(range(1, 13))
ax.set_xticklabels(meses_labels)

# Configurar Eje Y
ax.set_ylim(0, 100) 
ax.grid(True, alpha=0.5)

# Posicionar la leyenda fuera del área de trazado
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), title="Año")

plt.tight_layout()
plt.show()

# NEgados

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar estilo general
sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

# Extraer mes y año de Fecha_Proceso
df_ns_neg['Mes'] = df_ns_neg['Fecha_Proceso'].dt.month
df_ns_neg['Año'] = df_ns_neg['Fecha_Proceso'].dt.year
df_ns_neg['Año_Mes'] = df_ns_neg['Fecha_Proceso'].dt.to_period('M')

# Crear tabla de registros por mes y año
registros_mes_año = pd.crosstab(df_ns_neg['Mes'], df_ns_neg['Año'], margins=True, margins_name='Total')
print("Registros por mes y año:")
print(registros_mes_año)

# Definir paleta de colores personalizada
colores = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']

# Crear gráfica de líneas mejorada
plt.figure(figsize=(16, 8))
ax = plt.gca()

# Graficar cada año con estilo mejorado
for idx, año in enumerate(sorted(df_ns['Año'].unique())):
    datos_año = df_ns[df_ns['Año'] == año].groupby('Mes').size()
    plt.plot(datos_año.index, datos_año.values, 
             marker='o', label=f'{año}', linewidth=2.5, 
             color=colores[idx % len(colores)], markersize=8,
             markeredgewidth=2, markeredgecolor='white')

plt.xlabel('Mes', fontsize=13, fontweight='bold')
plt.ylabel('Cantidad de Registros', fontsize=13, fontweight='bold')
plt.title('Comportamiento Mensual de Registros por Año', 
          fontsize=16, fontweight='bold', pad=20)
plt.legend(title='Año', fontsize=11, title_fontsize=12, 
           frameon=True, shadow=True, fancybox=True)
plt.xticks(range(1, 13), ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 
                          'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'],
           fontsize=11)
plt.yticks(fontsize=11)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(True, alpha=0.2, linestyle='--')
plt.tight_layout()
plt.show()

# Crear heatmap mejorado
plt.figure(figsize=(14, 9))

# Preparar datos sin totales para el heatmap
datos_heatmap = registros_mes_año.iloc[:-1, :-1]

# Crear heatmap con mejor estética
ax = sns.heatmap(datos_heatmap, annot=True, fmt='d', 
                 cmap='RdYlGn_r', linewidths=0.5, linecolor='white',
                 cbar_kws={'label': 'Cantidad de Registros', 'shrink': 0.8},
                 annot_kws={'size': 11, 'weight': 'bold'},
                 xticklabels=datos_heatmap.columns,
                 yticklabels=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 
                             'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])

plt.xlabel('Año', fontsize=13, fontweight='bold', labelpad=10)
plt.ylabel('Mes', fontsize=13, fontweight='bold', labelpad=10)
plt.title('Mapa de Calor: Registros por Mes y Año', 
          fontsize=16, fontweight='bold', pad=20)
plt.xticks(fontsize=11, rotation=0)
plt.yticks(fontsize=11, rotation=0)

# Mejorar colorbar
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=10)
cbar.set_label('Cantidad de Registros', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()